In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from transformers import BertTokenizer, BertModel
import math

In [67]:
class attention(nn.Module):
    def __init__(self, embed_size, num_heads):
        super(attention, self).__init__()
        self.embed_size=embed_size
        self.num_heads=num_heads
        self.head_dim= embed_size // num_heads

        assert(self.head_dim * num_heads == embed_size), "Embed size must be completely divisible by number of heads"

        self.query=nn.Linear(self.head_dim,self.head_dim,bias=False)
        self.key=nn.Linear(self.embed_size,self.head_dim,bias=False)
        self.value=nn.Linear(self.embed_size,self.head_dim,bias=False)
        self.output=nn.Linear(num_heads*self.head_dim,embed_size)

    def forward(self,query,key,value,mask):
        batch_size=query.shape[0]
        q_len,k_len,v_len=query.shape[1],key.shape[1],value.shape[1]
        print("\n\n\nThe shapes before permutetion")
        query=query.reshape(batch_size,q_len,self.num_heads,self.head_dim)
        print("The shape of query is "+str(query.shape))
        key=key.reshape(batch_size,k_len,self.num_heads,self.head_dim)
        print("The shape of key is "+str(key.shape))
        value=value.reshape(batch_size,v_len,self.num_heads,self.head_dim)
        print("The shape of value is "+str(value.shape))
        # Transpose to perform batch matrix multiplication
        query = query.permute(0, 2, 1, 3)  # (batch_size, num_heads, q_len, head_dim)
        key = key.permute(0, 2, 1, 3)  # (batch_size, num_heads, k_len, head_dim)
        value = value.permute(0, 2, 1, 3)  # (batch_size, num_heads, v_len, head_dim)
        print("\n\n\nThe shapes after permutetion")
        print("The shape of query is "+str(query.shape))
        print("The shape of key is "+str(key.shape))
        print("The shape of value is "+str(value.shape))
        # Calculate attention scores
        scores = torch.matmul(query, key.transpose(-2,-1)) / torch.sqrt(torch.tensor(self.head_dim, dtype=torch.float32))

        # Apply mask if provided
        if mask is not None:
            mask1=torch.full(scores.size(),float('-inf'))
            #print(mask1)
            mask1=torch.triu(mask1,diagonal=1)
            #print(mask1)
            #print(scores)
            scores += mask1
            #print(scores)
        
        # Apply softmax to obtain attention weights
        attention_weights = torch.softmax(scores, dim=-1)

        # Apply dropout if needed (you can add this if desired)
        # attention_weights = self.dropout(attention_weights)

        # Apply attention weights to the values
        output = torch.matmul(attention_weights, value)
        print("The initial shape of output is "+str(output.shape))
        # Reshape and concatenate heads
        output = output.permute(0, 2, 1, 3).contiguous()  # (batch_size, q_len, num_heads, head_dim)
        output = output.reshape(batch_size, q_len, self.num_heads * self.head_dim)
        
        # Linear transformation to get the final output
        output = self.output(output)
        print("The final shape of output is "+str(output.shape))
        return output        
         

In [68]:
import torch
import torch.nn as nn

# Instantiate the Attention class
embed_size = 8
num_heads = 2
attention_model = attention(embed_size, num_heads)

# Define a sample input
batch_size = 2
q_len = 3
k_len = 3
v_len = 3

# Create random tensors as input
query = torch.randn(batch_size, q_len, embed_size)
key = torch.randn(batch_size, k_len, embed_size)
value = torch.randn(batch_size, v_len, embed_size)

# Define a sample mask (you can modify this based on your use case)
mask = 1  # Assuming a fully-connected attention

# Call the forward method
output = attention_model.forward(query, key, value, mask)

# Print the output shape
#print("Output Shape:", output)





The shapes before permutetion
The shape of query is torch.Size([2, 3, 2, 4])
The shape of key is torch.Size([2, 3, 2, 4])
The shape of value is torch.Size([2, 3, 2, 4])



The shapes after permutetion
The shape of query is torch.Size([2, 2, 3, 4])
The shape of key is torch.Size([2, 2, 3, 4])
The shape of value is torch.Size([2, 2, 3, 4])
The initial shape of output is torch.Size([2, 2, 3, 4])
The final shape of output is torch.Size([2, 3, 8])


In [69]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size,num_heads,dropout,foward_expansion):
        super(TransformerBlock,self).__init__()
        self.attention=attention(embed_size,num_heads=num_heads)
        self.norm1=nn.LayerNorm(embed_size)
        self.norm2=nn.LayerNorm(embed_size)

        self.feed_foward=nn.Sequential(
            nn.Linear(embed_size,foward_expansion*embed_size),
            nn.ReLU(),
            nn.Linear(foward_expansion*embed_size,embed_size)
        )

        self.dropout=nn.Dropout(dropout)

    def forward(self,query,key,value,mask):
        multi_head_attention=self.attention(query,key,value,mask)
        x=self.dropout(self.norm1(multi_head_attention+query))
        foward=self.feed_foward(x)  
        out=self.dropout(self.norm2(foward + x))
        return out


In [70]:
class PositionalEncoding1(nn.Module):
    def __init__(self, max_len, embed_size):
        super(PositionalEncoding1, self).__init__()
        self.embed_size = embed_size

        # Create constant positional encoding matrix
        pe = torch.zeros(max_len, embed_size)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_size, 2).float() * (-math.log(10000.0) / embed_size))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # Add batch dimension
        self.register_buffer('pe', pe)

    def forward(self, x):
        # Ensure that positional encoding has the same shape as input tensor x
        batch_size, seq_len = x.shape[:2]
        pe = self.pe[:, :seq_len, :].expand(batch_size, -1, -1)
        return pe


In [71]:
class Encoder(nn.Module):
    def __init__(self,
                 src_vocab_size,
                 embed_size,
                 num_layers,
                 num_heads,
                 device,
                 foward_expansion,
                 dropout,
                 max_length 
                ):
        super(Encoder,self).__init__()
        self.embed_size=embed_size
        self.device=device
        self.input_embedding=nn.Embedding(src_vocab_size,embed_size)
        self.positional_encoding=PositionalEncoding1(max_length,embed_size)

        self.layers=nn.ModuleList(
            [
                TransformerBlock(embed_size,num_heads,dropout,foward_expansion)
                for _ in range(num_layers)
            ]
        )
        self.dropout=nn.Dropout(dropout)
    
    def forward(self,x,mask):
        batch_size,seq_len=x.shape
        print("Batch size is : "+str(batch_size))
        print("Seq length is : "+str(seq_len)),

        embedding=self.input_embedding(x)
        #print("input embedding is :\n\n\n "+str(embedding)),
        print("\n\n\nInput embedding shape is :\n\n\n "+str(embedding.shape)),
        positions=torch.arange(0,seq_len).expand(batch_size,seq_len).to(device=self.device)
        #print("Position is :\n\n\n "+str(positions)),
        #print("\n\n\nPosition shape is :\n\n\n "+str(positions.shape)),
        # Generate positional encoding dynamically based on the input sequence length
        positional_encoding = self.positional_encoding(x)
        #positional_encoding = positional_encoding.unsqueeze(0).expand(1,batch_size, embed_size, embed_size)
        #print("Positional encoding is :\n\n\n "+str(positional_encoding)),
        print("\n\n\nPositional encoding shape is :\n\n\n "+str(positional_encoding.shape)),
        
        #out=self.dropout(self.input_embedding(x)+self.positional_encoding(positions))
        out=self.dropout(embedding+positional_encoding)

        for layer in self.layers:
            out=layer(out,out,out,mask)
        return out    




In [73]:
# Toy dataset (replace with your dataset)
src_vocab_size = 200  # Example vocabulary size
max_length = 20  # Example maximum sequence length
num_layers = 8
embed_size = 128
num_heads = 8
forward_expansion = 4
dropout = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the encoder
encoder = Encoder(src_vocab_size, embed_size, num_layers, num_heads, device, forward_expansion, dropout, max_length)

# Generate some sample input data
sample_input = torch.randint(0, src_vocab_size, (2, max_length))  # Batch size 2, sequence length 20
sample_mask = torch.ones_like(sample_input)

# Pass the input data through the encoder
encoder_output = encoder(sample_input, sample_mask)

# Check the output shape
print("Encoder output shape:", encoder_output.shape)

Batch size is : 2
Seq length is : 20



Input embedding shape is :


 torch.Size([2, 20, 128])



Positional encoding shape is :


 torch.Size([2, 20, 128])



The shapes before permutetion
The shape of query is torch.Size([2, 20, 8, 16])
The shape of key is torch.Size([2, 20, 8, 16])
The shape of value is torch.Size([2, 20, 8, 16])



The shapes after permutetion
The shape of query is torch.Size([2, 8, 20, 16])
The shape of key is torch.Size([2, 8, 20, 16])
The shape of value is torch.Size([2, 8, 20, 16])
The initial shape of output is torch.Size([2, 8, 20, 16])
The final shape of output is torch.Size([2, 20, 128])



The shapes before permutetion
The shape of query is torch.Size([2, 20, 8, 16])
The shape of key is torch.Size([2, 20, 8, 16])
The shape of value is torch.Size([2, 20, 8, 16])



The shapes after permutetion
The shape of query is torch.Size([2, 8, 20, 16])
The shape of key is torch.Size([2, 8, 20, 16])
The shape of value is torch.Size([2, 8, 20, 16])
The initial shape of 

In [42]:
sample_input.shape

torch.Size([2, 128])